<a href="https://colab.research.google.com/github/a-mhamdi/jlai/blob/main/Codes/Julia/Part-3/cnn/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HANDWRITTEN DIGITS RECOGNITION USING CNN
---

Handwritten digits classification using **CNN**. This solution is implemented in `Julia` using the `Flux.jl` library

In [1]:
versioninfo()

Julia Version 1.10.9
Commit 5595d20a287 (2025-03-10 12:51 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.00GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, skylake-avx512)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/lib64-nvidia
  JULIA_NUM_THREADS = auto


In [2]:
;wget https://raw.githubusercontent.com/a-mhamdi/jlai/refs/heads/main/Codes/Julia/Part-3/cnn/Project.toml

--2025-03-27 13:35:42--  https://raw.githubusercontent.com/a-mhamdi/jlai/refs/heads/main/Codes/Julia/Part-3/cnn/Project.toml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 707 [text/plain]
Saving to: ‘Project.toml’

     0K                                                       100% 33.1M=0s

2025-03-27 13:35:42 (33.1 MB/s) - ‘Project.toml’ saved [707/707]



In [3]:
;sed -i '/^Pluto/d' Project.toml

In [ ]:
import Pkg; Pkg.activate("."); Pkg.instantiate(); Pkg.precompile(); Pkg.resolve();
Pkg.status()

  Activating project at `/content`
    Updating registry at `~/.julia/registries/General.toml`
   Installed iso_codes_jll ───────────── v4.15.1+0
   Installed ImageSegmentation ───────── v1.8.1
   Installed ImageIO ─────────────────── v0.6.8
   Installed ContextVariablesX ───────── v0.1.3
   Installed TiledIteration ──────────── v0.5.0
   Installed ShowCases ───────────────── v0.1.0
   Installed Accessors ───────────────── v0.1.42
   Installed TiffImages ──────────────── v0.10.2
   Installed HistogramThresholding ───── v0.3.1
   Installed ZipFile ─────────────────── v0.10.1
   Installed Images ──────────────────── v0.26.1
   Installed ImageMagick ─────────────── v1.2.1
   Installed NearestNeighbors ────────── v0.4.21
   Installed RegionTrees ─────────────── v0.3.2
   Installed Cairo_jll ───────────────── v1.18.2+1
   Installed BFloat16s ───────────────── v0.5.1
   Installed MetaGraphs ──────────────── v0.7.2
   Installed InitialValues ───────────── v0.3.1
   Installed Polynomials ─────

Import the machine learning library `Flux`

In [ ]:
using Flux
using Flux: DataLoader
using Flux: onecold, onehotbatch

In [ ]:
using CUDA
CUDA.versioninfo()

In [ ]:
Base.@kwdef mutable struct HyperParams
    η = 3f-3                # Learning rate
    batchsize = 64          # Batch size
    epochs = 8              # Number of epochs
    split = :train          # Split data into `train` and `test`
end

Load the **MNIST** dataset

In [ ]:
using MLDatasets

In [ ]:
d = MNIST()

In [ ]:
function get_data(; kws...)
    args = HyperParams(; kws...);
    # Split and normalize data
    data = MNIST(split=args.split);
    X, y = data.features ./ 255, data.targets;
    X = reshape(X, (28, 28, 1, :));
    y = onehotbatch(y, 0:9);
    loader = DataLoader((X, y); batchsize=args.batchsize, shuffle=true) |> gpu;
    return loader
end

In [ ]:
train_loader = get_data();
test_loader = get_data(split=:test);

Transform sample training data to an image. View the image and check the corresponding digit value.

In [ ]:
using Statistics

In [ ]:
idx = rand(1:6_000, 3)

In [ ]:
using ImageShow, ImageInTerminal

In [ ]:
convert2image(d, idx)

In [ ]:
"Digit are $(d.targets[idx])"

**CNN** ARCHITECTURE

The input `X` is a batch of images with dimensions `(width=28, height=28, channels=1, batchsize)`

In [ ]:
fc = prod(Int.(floor.([28/4 - 2, 28/4 - 2, 16]))) # 2^{\# max-pool}

In [ ]:
model = Chain(
            Conv((5, 5), 1 => 16, relu),  # (28-5+1)x(28-5+1)x16 = 24x24x16
            MaxPool((2, 2)),              # 12x12x16
            Conv((3, 3), 16 => 16, relu), # (12-3+1)x(12-3+1)x16 = 10x10x16
            MaxPool((2, 2)),              # 5x5x16
            Flux.flatten,                 # 400
            Dense(fc => 64, relu),
            Dense(64 => 32, relu),
            Dense(32 => 10)
) |> gpu

In [ ]:
using ProgressMeter: Progress, next!

In [ ]:
function train(; kws...)
    args = HyperParams(; kws...)
    # Define the loss function
    l(α, β) = Flux.logitcrossentropy(α, β)
    # Define the accuracy metric
    acc(α, β) = mean(onecold(α) .== onecold(β))
    # Optimizer
    optim_state = Flux.setup(Adam(args.η), model);

    vec_loss = []
    vec_acc = []

    for epoch in 1:args.epochs
        printstyled("\t***\t === EPOCH $(epoch) === \t*** \n", color=:magenta, bold=true)
        @info "TRAINING"
        prg_train = Progress(length(train_loader))
        for (X, y) in train_loader
            loss, grads = Flux.withgradient(model) do m
                ŷ = m(X);
                l(ŷ, y);
            end
            Flux.update!(optim_state, model, grads[1]); # Upd `W` and `b`
            # Show progress meter
            next!(prg_train, showvalues=[(:loss, loss)])
        end
        @info "TESTING"
        prg_test = Progress(length(test_loader))
        for (X, y) in test_loader
            ŷ = model(X);
            push!(vec_loss, l(ŷ, y));  # log `loss` value -> `vec_loss` vector
            push!(vec_acc, acc(ŷ, y)); # log `accuracy` value -> `vec_acc` vector
          	# Show progress meter
            next!(prg_test, showvalues=[(:loss, vec_loss[end]), (:accuracy, vec_acc[end])])
        end
    end
    return vec_loss, vec_acc
end

In [ ]:
vec_loss, vec_acc = train()

Plot results

In [ ]:
using Plots

In [ ]:
plot(vec_loss, label="Test Loss")
plot(vec_acc, label="Test Accuracy")

Let's make some predictions

In [ ]:
idx = rand(1:1000, 16)
xs = test_loader.data[idx][1]
yp = xs |> model |> softmax |> out -> onecold(out, 0:9) |> cpu
ys = onecold(test_loader.data[idx][2]) .- 1 |> cpu;

In [ ]:
for i ∈ eachindex(yp)
    @info "**Prediction is $(yp[i]). Label is $(ys[i]).**"
end

Save the model

In [ ]:
#=
using BSON: @save
@save "cnn.bson" model
=#